## Using this notebook
 
- You have to execute this notebook by running each cell consecutively.
- To retain any changes made, you need to save the notebook. 

For detailed information about the notebook interface see _Help -> User Interface Tour_ in the menu, and the [Guidebook](https://lbologna.github.io/hbp-sp6-guidebook/getting_started/getting_started.html#working-with-collabs) for working with Collabs.

In [1]:
!pip install -q -i https://bbpteam.epfl.ch/repository/devpi/simple notebook-version-check
notebook_id = '94569d862a2197896a62f0f822ce2766cca01e66'
notebook_url = 'https://api.github.com/repos/antonelepfl/usecases/contents/production_notebooks/singlecellmodeling/Synaptic%20plasticity.ipynb?ref=master'
from notebookchecklib.nbversioncheck import NBVersionCheck
check = NBVersionCheck(notebook_id, notebook_url, get_hbp_service_client(), get_collab_storage_path())
check.check()

In [ ]:
from IPython.display import display, HTML

REQUIRED_VERSIONS = {
    "hbp_service_client": "1.1.0",
    'ipywidgets':'7.5.1'
}

def get_version(package_name):
    """Returns version for the given package"""
    if package_name=='hbp_service_client':
        import hbp_service_client
        return hbp_service_client.__version__
    if package_name=='ipywidgets':
        import ipywidgets
        return '.'.join(map(str, ipywidgets.version_info[0:3]))
    
def install_package(package_name, versions):
    """Updates the specified package."""
    print("Changing package {}".format(package_name))
    if package_name=='hbp_service_client':
        lib = "{}=={}".format(package_name, versions[package_name])
        !pip install -I --quiet $lib     
    if package_name=='ipywidgets':
        lib1 = 'ipywidgets=={}'.format(versions["ipywidgets"])
        versions["widgetsnbextension"]='3.5.1'
        lib2 = 'widgetsnbextension=={}'.format(versions["widgetsnbextension"])
        !pip2 install $lib1 $lib2
        !pip3 install $lib1 $lib2
        !jupyter nbextension uninstall --py widgetsnbextension
        !jupyter nbextension install --py --user widgetsnbextension
        !jupyter nbextension enable widgetsnbextension --user --py
    
# Check if a package needs to be changed
reload_required = []
for package_name, required_version in REQUIRED_VERSIONS.items():
    if get_version(package_name) != required_version:
        print("{} {}".format(get_version(package_name), required_version))
        reload_required.append(package_name)
        
if reload_required:
    display(HTML("""
    <div class="alert alert-block alert-danger">Some internal libraries had to be changed...<br>
    Please follow the popup to reload this page.</div>"""))
    
    # Change all package that requires change
    for package in reload_required:
        install_package(package, REQUIRED_VERSIONS)
    
    display(HTML("""<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
    Jupyter.notebook.dirty = false; window.location.reload(); })</script>"""))
else:
    display(HTML("""
    <div class="alert alert-block alert-success">Setup for this notebook is correct.</div>"""))

In [ ]:
!rm -f /home/jovyan/.local/nrn
!ln -s /home/jovyan/.local/nrn-7.6 /home/jovyan/.local/nrn

In [ ]:
model_url = 'CA1_pyr_cACpyr_oh140807_A0_idG_20190328143451'

In [ ]:
%%capture
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [ ]:
from ipywidgets import widgets
from IPython.display import *
from ipywidgets import *
from IPython.display import HTML,Javascript, display, clear_output
import pprint
import os
import shutil
from ipywidgets import *
import json
import io
import zipfile
os.chdir("/home/jovyan")
if os.path.isdir('GUIPLAST')==False:
    !mkdir GUIPLAST
else:
    shutil.rmtree('GUIPLAST')
    !mkdir GUIPLAST
os.chdir("GUIPLAST")
#display(HTML('''<style>
#    .widget-label { min-width: 17ex !important;}
#</style>'''))

javascript2 = """
<script type="text/Javascript">
document.styleSheets[0].insertRule(".widget-listbox.form-control { width: auto }", 1);
document.styleSheets[0].insertRule(".form_login > label { width: 30% }", 1);
document.styleSheets[0].insertRule(".form_login > label + input { width: 70% }", 1);
</script>
"""
h1=HTML(javascript2)
h1.display='none'
#display(h1)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
clients = get_hbp_service_client()
foldernameOPTstring = model_url
clients = get_hbp_service_client()
remote_storage = '70842'
listfilenames=[]
for filename in os.listdir(os.getcwd()):
    listfilenames.append(filename)
if 'STDPE2Syn2.mod' not in listfilenames:
    clients.storage.download_file(os.path.join(os.path.sep,remote_storage,\
                                               'STDPE2Syn2.mod'),\
                                  'STDPE2Syn2.mod');
if 'ProbGABAAB_EMS_GEPH_g.mod' not in listfilenames:
    clients.storage.download_file(os.path.join(os.path.sep,remote_storage,\
                                               'ProbGABAAB_EMS_GEPH_g.mod'), \
                                  'ProbGABAAB_EMS_GEPH_g.mod');
if 'axis3dwithneuron_new3.py' not in listfilenames:
    clients.storage.download_file(os.path.join(os.path.sep,remote_storage,\
                                               'axis3dwithneuron_new3.py'),\
                                  'axis3dwithneuron_new3.py');

In [ ]:
%%capture
URL=('https://object.cscs.ch/v1/AUTH_c0a333ecf7c045809321ce9d9ecdfdea/'
'hippocampus_optimization/'
'bsp_data_repository-4.0.5_Python3.tar.gz')
!wget "{URL}"
!tar -zxvf bsp_data_repository-4.0.5_Python3.tar.gz

In [ ]:
diff = lambda l1,l2: [x for x in l1 if x not in l2]
os.chdir('bsp_data_repository-4.0.5_Python3')
os.chdir('optimizations')
FD=diff(os.listdir('.'),['README'])
for i in range(len(FD)):
    shutil.copytree(FD[i],os.path.join('..','..',FD[i]))
os.chdir('..')
os.chdir('..')

In [ ]:
for files in os.listdir(foldernameOPTstring):
    if files.endswith('.zip'):
        os.chdir(foldernameOPTstring)
        zip_ref = zipfile.ZipFile(files, 'r')
        zip_ref.extractall()
        zip_ref.close()          
os.chdir('..')

def byteify(input):
    if isinstance(input, dict):
        return {byteify(key): byteify(value)
                for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [byteify(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input

In [ ]:
import plotly
plotly.offline.init_notebook_mode()
from plotly.offline import iplot, plot
from plotly.graph_objs import graph_objs as go
from plotly.graph_objs import Scatter, Layout
import shutil
import os
from shutil import copy2

In [ ]:
from __future__ import print_function # py 2.7 compat.
import base64
import ipywidgets as widgets # Widget definitions.
from traitlets import Unicode # Traitlet needed to add synced attributes to the widget.
file_widget = widgets.FileUpload() 

In [ ]:
import logging
logger = logging.getLogger()
logging.disable(logging.WARNING)
import neurom as nm
os.chdir(foldernameOPTstring)
with open(foldernameOPTstring+'_meta.json') as json_file:
    y = json.load(json_file)
best_cell=y["best_cell"]
os.chdir(foldernameOPTstring)

def plot_to_html(plt):
    # write image data to a string buffer and get the PNG image bytes
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return """<img src='data:image/png;base64,{}'/>""".format(base64.b64encode(buf.getvalue()).decode('ascii'))

os.chdir('morphology')
for files in os.listdir('.'):
    nrn = nm.load_neuron(files);
from neurom import viewer
import matplotlib
import numpy as np
plt,ax=viewer.draw(nrn, mode='2d');
outputHTML = widgets.HTML()
outputHTML.value = plot_to_html(plt)
os.chdir('..')

In [ ]:
from ipywidgets import widgets, HBox, Label, Layout
from IPython.display import display
from shutil import copy2
import sys
os.chdir('morphology')
copy2(os.path.join('..','..','..','axis3dwithneuron_new3.py'),'axis3dwithneuron_new3.py')

%matplotlib notebook
from matplotlib import pyplot as plt
import random
os.chdir('..')

file_widget.layout.display='none'

def displayTMwhtsimnr(_):
    listnr=[x for x in range(int(synnr.value))]
    TM.options=listnr
    if RBM.value=='default':
        if modfiles.value=='STDPE2Syn2.mod':
            TM.layout.display=''
        if modfiles.value=='ProbGABAAB_EMS_GEPH_g.mod':
            wht.layout.display='none'
    else:
        TM.layout.display=''
    if VCLAMPrec.value==True:
        RS.layout.display='none'
        TM.layout.display='none'
        simnr.layout.display=''
    
def modfile(change):
    if RBM.value=='default':
        modfiles.layout.display=''
        DESCR.layout.display=''
        file_widget.layout.display='none'
        LINK.layout.display=''
        wht.value='_ref_weight[1]'
        wht.disabled=True
        if modfiles.value=='STDPE2Syn2.mod':
            TM.layout.display=''
        if modfiles.value=='ProbGABAAB_EMS_GEPH_g.mod':
            wht.layout.display='none'
    else:
        modfiles.layout.display='none'
        DESCR.layout.display='none'
        file_widget.layout.display=''
        LINK.layout.display='none'
        wht.value=''
        wht.disabled=False
    displayTMwhtsimnr(_)
    
def modfile2(change):
    if modfiles.value=='STDPE2Syn2.mod':
        DESCR.value="Bianchi, D., De Michele, P., Marchetti, C., Tirozzi, B., Cuomo, S., Marie, H., Migliore, M. (2014). Effects of increasing CREB-dependent transcription on the storage and recall processes in a hippocampal CA1 microcircuit. Hippocampus. 24(2):165-77."
        LINK.layout.display=''
    else:
        DESCR.value="Lupascu, C. A., Morabito, A., Merenda, E., Marinelli, S., Marchetti, C., Migliore, R., Cherubini, E., Migliore, M. (2016). A General Procedure to Study Subcellular Models of Transsynaptic Signaling at Inhibitory Synapses. Frontiers in neuroinformatics, 10, 23."
        LINK.layout.display='none'
    displayTMwhtsimnr(_)   
    
RBM=widgets.RadioButtons(options=['default','upload'],description='mod file')
RBM.layout.display=''
RBM.observe(modfile,names='value')
modfiles=widgets.Dropdown(
    options=['STDPE2Syn2.mod','ProbGABAAB_EMS_GEPH_g.mod'],
    value='STDPE2Syn2.mod',
    description='MOD:',
    disabled=False,layout={'width': '95%'}
)
boxmodfiles= VBox([modfiles], layout={'width': '400px'})
modfiles.observe(modfile2,names='value')
DESCR=Textarea(layout={'height': '100%','width': '95%'})
DESCR.description='paper'
DESCR.value="Bianchi, D., De Michele, P., Marchetti, C., Tirozzi, B., Cuomo, S., Marie, H., Migliore, M. (2014). Effects of increasing CREB-dependent transcription on the storage and recall processes in a hippocampal CA1 microcircuit. Hippocampus. 24(2):165-77."
LINK=widgets.HTML("""<a href="https://www.ncbi.nlm.nih.gov/pubmed/24123649">https://www.ncbi.nlm.nih.gov/pubmed/24123649</a>""")
DESCR.disabled=True
boxdescr = VBox([DESCR], layout={'height': '150px','width':'400px'})
def expprotchange(change):
    if expprot.value=='frequency':
        netstimintv.layout.display=''
        labelnetstimintv.layout.display=''
        netstimnr.layout.display=''
        labelnetstimnr.layout.display=''
        SPACEL.layout.display='none'
        SPACE5.layout.display='none'
        SPACE6.layout.display='none'
        netstimintv1.layout.display='none'
        netstimnr1.layout.display='none'
        netstimintv2.layout.display='none'
        netstimnr2.layout.display='none'
        L5.layout.display='none'
        L6.layout.display='none'
        netstimnoise.value='0.5'
        SYNC.options=['sync','async']
    else:
        netstimintv.layout.display='none'
        labelnetstimintv.layout.display='none'
        netstimnr.layout.display='none'
        labelnetstimnr.layout.display='none'
        SPACEL.layout.display=''
        SPACE5.layout.display=''
        SPACE6.layout.display=''
        netstimintv1.layout.display=''
        netstimnr1.layout.display=''
        netstimintv2.layout.display=''
        netstimnr2.layout.display=''
        L5.layout.display=''
        L6.layout.display=''
        netstimnoise.value='0'
        SYNC.options=['sync']
        
expprot=widgets.Dropdown(
    options=['frequency', 'theta burst'],
    value='frequency',
    description='exp protocol:',
    disabled=False,
)
expprot.on_trait_change(expprotchange)
tstop=Text(description='tstop',value='600',width="20%")
synnr=Text(value='10',width='20%')
labelsynnr=widgets.Label('number',layout=Layout(width='10%'))
mindist=Text(value='50',width='20%')
labelmindist=widgets.Label('min distance',layout=Layout(width='10%'))
maxdist=Text(value='300',width='20%')
labelmaxdist=widgets.Label('max distance',layout=Layout(width='10%'))
SYNC=widgets.RadioButtons(options=['sync','async'])
SYNC.layout.display=''
netstimintv=Text(value='50',width="20%")
labelnetstimintv=widgets.Label('Netstim freq',layout=Layout(width='13%'))
netstimintv1=Text(value='100',width="20%")
netstimintv1.layout.display='none'
netstimintv2=Text(value='4',width="20%")
netstimintv2.layout.display='none'
netstimnr=Text(value='10000',width="20%")
labelnetstimnr=widgets.Label('Netstim nr',layout=Layout(width='13%'))
netstimnr1=Text(value='4',width="20%")
netstimnr1.layout.display='none'
netstimnr2=Text(value='10',width="20%")
netstimnr2.layout.display='none'
netstimstart=Text(value='200',width="20%")
labelnetstimstart=widgets.Label('Netstim start',layout=Layout(width='13%'))
netstimnoise=Text(value='0.5',width="20%")
if expprot.value=='frequency':
    netstimnoise.value='0.5'
else:
    netstimnoise.value='0'
labelnetstimnoise=widgets.Label('Netstim noise',layout=Layout(width='13%'))
netconweight=Text(value='1e-3',width="20%")
labelnetconweight=widgets.Label('Netcon weight',layout=Layout(width='13%'))

stimdelay=Text(value='100',width="20%")
stimdur=Text(value='300',width="20%")
stimamp=Text(value='0',width="20%")
stimrep=Text(value='0',width="20%")
vclampvrev=Text(value='-17',width="20%")
vclampamp=Text(value='-70',width="20%")
vclampdur=Text(width="20%")
vclampdur.value=tstop.value
APIC=Checkbox(value=False,disabled=True,indent=False)
if model_url[model_url.find('CA1_')+4:model_url.find('CA1_')+7]=='pyr':
    APIC.value=True
    APIC.disabled=False
APIC.layout.width="10%"
labelAPIC=widgets.Label('apical',layout=Layout(width='10%'))
DEND=Checkbox(value=True,disabled=False,indent=False)
DEND.layout.width="10%"
labelDEND=widgets.Label('dendrites',layout=Layout(width='10%'))
VCLAMPrec=Checkbox(value=False,disabled=False,indent=False)
VCLAMPrec.layout.width="10%"
labelVCLAMP=widgets.Label('vclamp recording',layout=Layout(width='20%'))
simnr=Text(description='nr of simulations',value='10',width='20%',style={'description_width': 'initial'})
simnr.layout.display=''
wht=Text(description='variable to record',value='_ref_weight[1]',width="30%",style={'description_width': 'initial'})
wht.disabled=True
TM=widgets.SelectMultiple(
    options=[0],
    value=[0],
    description='synapses to record from',
    disabled=False,
    style={'description_width': 'initial'}
)
TM.layout.display='none'
displayTMwhtsimnr(_)

def changed(b):
    if VCLAMPrec.value==True:
        TM.layout.display='none'
        RS.layout.display='none'
    else:
        TM.layout.display=''
        RS.layout.display=''
    simnr.layout.display=''

VCLAMPrec.observe(changed)
global firsttimeplacesyn
firsttimeplacesyn=1
out1 = widgets.Output()

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

import sys

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

def loadcell1(_):
    os.chdir('mechanisms')
    if RBM.value=='default':
        copy2(os.path.join('..','..','..','STDPE2Syn2.mod'),'STDPE2Syn2.mod')
        copy2(os.path.join('..','..','..','ProbGABAAB_EMS_GEPH_g.mod'),\
              'ProbGABAAB_EMS_GEPH_g.mod')
    else:
        with open(list(file_widget.value.keys())[0], "wb") as fp:
            fp.write(file_widget.value[list(file_widget.value.keys())[0]]['content'])
    os.chdir('..')
    if os.path.isdir('x86_64/')==True:
        shutil.rmtree('x86_64')
    myvar=!nrnivmodl mechanisms
    import neuron
    os.chdir('checkpoints')
    from neuron import h
    h.load_file('stdrun.hoc')
    h.celsius=35
    h.load_file(best_cell)
    with open(best_cell) as ff:
        searchlines=ff.readlines()
    for kk, line in enumerate(searchlines):
        if "begintemplate" in line:
            break
    l2=line.split()
    global templatename
    templatename=l2[1]
    os.chdir('..')
    
def loadcell2(_):
    from neuron import h
    global testcell
    testcell = eval('h.'+templatename+'(\'morphology\')');
    h.init()
    
def placesyn(_):
    loadcell1(_)
    with Capturing() as output:
        loadcell2(_);
    from neuron import h
    hasapic=False
    sectionnames=[]
    for sec in testcell.all:
        if 'apic' in sec.name()[sec.name().find('].')+2:100]:
            hasapic=True
        sectionnames.append(sec.name()[sec.name().find('].')+2:100])
    segms=[]
    for seg in testcell.all:
        segms.append(seg.name()[seg.name().find('.')+1:100])
    RS.options=segms
    RS.value=segms[0]
    global firsttimeplacesyn
    if firsttimeplacesyn==1:
        firsttimeplacesyn=2
    #clear_output()
    global listall
    if APIC.value==True:
        nrapicseg=0
        for nrapic in range(len(testcell.apic)):
            nrapicseg=nrapicseg+testcell.apic[nrapic].nseg
    if DEND.value==True:
        nrdendseg=0
        for nrdend in range(len(testcell.dend)):
            nrdendseg=nrdendseg+testcell.dend[nrdend].nseg
    
    listapicseg2=[]
    if APIC.value==True:
        for seg in testcell.soma[0]:
            h.distance(0,seg.x, sec=testcell.soma[0])
        nrapicseg2=0
        for nrapic in range(len(testcell.apic)):
            for seg in testcell.apic[nrapic]:
                if h.distance(seg.x,sec=testcell.apic[nrapic])>int(mindist.value) \
                and h.distance(seg.x,sec=testcell.apic[nrapic])<int(maxdist.value):
                    nrapicseg2=nrapicseg2+1
                    listapicseg2.append([testcell.apic[nrapic].name(), seg.x])
    listdendseg2=[]
    if DEND.value==True:
        nrdendseg2=0
        for seg in testcell.soma[0]:
                h.distance(0,seg.x, sec=testcell.soma[0])
        for nrdend in range(len(testcell.dend)):
            for seg in testcell.dend[nrdend]:
                if h.distance(seg.x,sec=testcell.dend[nrdend])>int(mindist.value) \
                and h.distance(seg.x,sec=testcell.dend[nrdend])<int(maxdist.value):
                    nrdendseg2=nrdendseg2+1
                    listdendseg2.append([testcell.dend[nrdend].name(), seg.x])
    listall=listapicseg2+listdendseg2
    global index
    if len(listall)<int(synnr.value):
        index=random.sample(range(len(listall)),len(listall))
    else:
        index=random.sample(range(len(listall)),int(synnr.value))
    global pozs
    pozs=[]
    for i in range(len(index)):
        pozs.append(eval('testcell.'+listall[index[i]][0][listall[index[i]][0].find('].')+\
                                                   2:len(listall[index[i]][0])]))
    outputHTML.layout.display='none'
    os.chdir('morphology')
    from axis3dwithneuron_new3 import Axis3DWithNEURON
    os.chdir('..')
    with out1:
        clear_output()
        fig = plt.figure(figsize=(6, 6))
        ax = Axis3DWithNEURON(fig)
        ax.grid(False)
        ax.axis('off')
        ax.shapeplot(variable='v')
        for i in range(len(pozs)):
            ax.mark(eval('testcell'+pozs[i].name()[pozs[i].name().find('].')+1:100]+\
                         '('+str(listall[i][1])+')'),marker='oc',number=i)
        ax.auto_aspect()
    listnr=[x for x in range(int(synnr.value))]
    TM.options=listnr
    if VCLAMPrec.value==True:
        RS.layout.display='none'
        TM.layout.display='none'
        simnr.layout.display=''
        
SYN = widgets.Button()
SYN.description = 'Place synapses'
SYN.background_color='gainsboro'
SYN.border_color='black'
SYN.width='81%'
SYN.on_click(placesyn)

def movetostorage(_):
    clients = get_hbp_service_client()
    storeto_path = os.path.join(get_collab_storage_path(),'plasticity_traces')
    if not clients.storage.exists(storeto_path):
        clients.storage.mkdir(storeto_path)
    clients.storage.upload_file(expprot.value+SYNC.value+timestampStr+'.csv',\
                                str(os.path.join(storeto_path,\
                                                 expprot.value+SYNC.value+timestampStr+'.csv'\
                                                )),'txt');
    Rec2.disabled=True

def myadvance():
    from neuron import h
    h.load_file('stdrun.hoc')
    while h.t < h.tstop:
        sys.stdout.write('{:2f}'.format(h.t)+'\r')
        sys.stdout.flush()
        h.fadvance()

import csv
from datetime import datetime

def startcounting():
    from neuron import h
    h.load_file('stdrun.hoc')
    h('proc advance() {nrnpython("myadvance()")}')

def play_prot(_):
    from neuron import h
    startcounting()
    #clear_output()
    with out2:
        clear_output()
    h.finitialize()
    h.init()
    os.chdir('mechanisms')
    if RBM.value=='default':
        if modfiles.value=='STDPE2Syn2.mod':
            with open('STDPE2Syn2.mod') as ff:
                searchlines=ff.readlines()
        if modfiles.value=='ProbGABAAB_EMS_GEPH_g.mod':
            with open('ProbGABAAB_EMS_GEPH_g.mod') as ff:
                searchlines=ff.readlines()
    else:
        with open(list(file_widget.value.keys())[0]) as ff:
            searchlines=ff.readlines()
    for kk, line in enumerate(searchlines):
        if "POINT_PROCESS" in line:
            break
    l2=line.split()
    os.chdir('..')
    global syn,netcon
    syn=[]
    netcon=[]
    netstim=[]
    neuron.h.mcell_ran4_init(1234567)
    synapse_rng = neuron.h.Random()
    synapse_rng.MCellRan4(1234567)
    synapse_rng.uniform(0, 1)
    synapse_delay = neuron.h.Random()
    synapse_delay.MCellRan4(1234567)
    synapse_delay.uniform(0, 2)
    if expprot.value=='frequency':
        if SYNC.value=='async':
            for i in range(len(index)):
                netstim.append(h.NetStim(0.5,sec=testcell.soma[0]))
                netstim[-1].interval = 1000/float(netstimintv.value)
                netstim[-1].number = float(netstimnr.value)
                netstim[-1].start = float(netstimstart.value)+synapse_delay.uniform(0, 2)
                netstim[-1].noise = float(netstimnoise.value)
        
                syn.append(h.__getattribute__(l2[1])(eval('testcell.'+\
                                                          listall[i][0][listall[i][0].find('].')+\
                                                                        2:len(listall[i][0])]+\
                                                          '('+str(listall[i][1])+')')))
                if RBM.value=='default':
                    if modfiles.value=='ProbGABAAB_EMS_GEPH_g.mod':
                        syn[-1].verboseLevel = 0
                        syn[-1].Use = 1.0
                        syn[-1].u0 = 1.0
                        syn[-1].e_GABAA = float(vclampvrev.value)
                        syn[-1].setRNG(synapse_rng)
                netcon.append(h.NetCon(netstim[-1], syn[-1]))
                netcon[-1].delay = 0
                netcon[-1].threshold = -20
                netcon[-1].weight[0] = float(netconweight.value)
        if SYNC.value=='sync':
            netstim.append(h.NetStim(0.5,sec=testcell.soma[0]))
            netstim[-1].interval = 1000/float(netstimintv.value)
            netstim[-1].number = float(netstimnr.value)
            netstim[-1].start = float(netstimstart.value)+synapse_delay.uniform(0, 2)
            netstim[-1].noise = float(netstimnoise.value)
            for i in range(len(index)):
                syn.append(h.__getattribute__(l2[1])(eval('testcell.'+\
                                                          listall[i][0][listall[i][0].find('].')+\
                                                                        2:len(listall[i][0])]+\
                                                          '('+str(listall[i][1])+')')))
                if RBM.value=='default':
                    if modfiles.value=='ProbGABAAB_EMS_GEPH_g.mod':
                        syn[-1].verboseLevel = 0
                        syn[-1].Use = 1.0
                        syn[-1].u0 = 1.0
                        syn[-1].e_GABAA = float(vclampvrev.value)
                        syn[-1].setRNG(synapse_rng)
                netcon.append(h.NetCon(netstim[-1], syn[-1]))
                netcon[-1].delay = 0
                netcon[-1].threshold = -20
                netcon[-1].weight[0] = float(netconweight.value)
    else:
        netstiminter = h.NetStim(0.5,sec=testcell.soma[0])
        netstiminter.interval = 1000/float(netstimintv2.value)
        netstiminter.number = float(netstimnr2.value)
        netstiminter.start = float(netstimstart.value)+synapse_delay.uniform(0, 2)
        netstiminter.noise = float(netstimnoise.value)
        netstimintra = h.NetStim(0.5,sec=testcell.soma[0])
        netstimintra.interval = 1000/float(netstimintv1.value)
        netstimintra.number = float(netstimnr1.value)
        netstimintra.start = float(netstimstart.value)+synapse_delay.uniform(0, 2)
        netstimintra.noise = float(netstimnoise.value)
        netconin = h.NetCon(netstiminter, netstimintra)
        netconin.delay = 0
        netconin.threshold = -20
        netconin.weight[0] = float(netconweight.value)
        for i in range(len(index)):
            syn.append(h.__getattribute__(l2[1])(eval('testcell.'+\
                                                      listall[i][0][listall[i][0].find('].')+\
                                                                    2:len(listall[i][0])]+\
                                                      '('+str(listall[i][1])+')')))
            if RBM.value=='default':
                    if modfiles.value=='ProbGABAAB_EMS_GEPH_g.mod':
                        syn[-1].verboseLevel = 0
                        syn[-1].Use = 1.0
                        syn[-1].u0 = 1.0
                        syn[-1].e_GABAA = float(vclampvrev.value)
                        syn[-1].setRNG(synapse_rng)
            netcon.append(h.NetCon(netstimintra, syn[-1]))
            netcon[-1].delay = 0
            netcon[-1].threshold = -20
            netcon[-1].weight[0] = float(netconweight.value)
    
    for i in range(int(stimrep.value)):
        stim = neuron.h.IClamp(0.5,testcell.soma)
        stim.delay=float(stimdelay.value)
        stim.amp=float(stimamp.value)
        stim.dur=float(stimdur.value)
    
    if VCLAMPrec.value==True:
        vclamp = neuron.h.VClamp(0.5, testcell.soma)
        vclamp.dur[0] = float(vclampdur.value)
        vclamp.amp[0] = float(vclampamp.value)
        h.cvode_active(0)
    else:
        h.cvode_active(1)
    h.tstop=float(tstop.value)
    h.CVode().re_init() 
    timeVector = h.Vector()
    timeVector.record(h._ref_t)
    
    if VCLAMPrec.value==True:
        ap = h.Vector()
        ap.record(vclamp._ref_i)  
    else:
        ap = h.Vector()
        ap.record(eval('testcell.'+RS.value+'(0.5)'+'._ref_v'),sec=eval('testcell.'+RS.value))
        ncweightsall={}
        ncweightsall1={}
        ncweightsall2={}
        for i in range(len(TM.value)):
            ncweightsall[i] = h.Vector()
            ncweightsall1[i] = h.Vector()
            ncweightsall2[i] = h.Vector()
            if RBM.value=='default':
                if modfiles.value=='STDPE2Syn2.mod':
                    ncweightsall[i].record(eval('netcon[TM.value[i]].'+wht.value),\
                                           sec=eval('testcell.'+RS.value))
            else:
                a=wht.value
                ncweightsall1[i].record(eval('syn[TM.value[i]]._ref_'+a[0:a.find(',')]))
                ncweightsall2[i].record(eval('syn[TM.value[i]]._ref_'+\
                                             a[a.find(',')+1:len(a)]))
    global model_current, timeVectorP
    layout = go.Layout(
    xaxis=dict(
        title='time (ms)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        hoverformat = '.2f'
    ))
    if VCLAMPrec.value==True:
        layout.yaxis['range']=[-1,0]
        model_currents=[]
        for i in range(int(simnr.value)):
            print("simulation %s" %str(int(i+1)))
            h.run()
            model_current = ap.to_python()
            timeVectorP=timeVector.to_python()
            model_currents.append(model_current)
        avg_current = [float(sum(col))/len(col) for col in zip(*model_currents)]
        #clear_output()
        layout.yaxis['title']='current (pA)'
        with out2:
            clear_output()
            traces=[]
            columns=[]
            columns.append(timeVectorP)
            for i in range(int(simnr.value)):
                trace = go.Scatter(x=timeVectorP,y=model_currents[i],mode = 'lines', \
                                   name = 'current trace '+ str(int(i+1)))
                traces.append(trace)
                columns.append(model_currents[i])
            trace = go.Scatter(x=timeVectorP,y=avg_current,mode = 'lines+markers', \
                               name = 'mean current trace ',\
                               line=dict(width=5))
            traces.append(trace)
            columns.append(avg_current)
            
            iplot(go.Figure(data=traces, layout=layout))
    else:
        h.run()
        if RBM.value=='default':
            for i in range(len(TM.value)):
                ncweightsall[i].mul(5000.0)
        else:
            for i in range(len(TM.value)):
                ncweightsall[i]=ncweightsall1[i].sub(ncweightsall2[i])
        model_current = ap.to_python()
        timeVectorP=timeVector.to_python()
        ncweightsP={}
        for i in range(len(TM.value)):
            ncweightsP[i]=ncweightsall[i].to_python()
        #clear_output()
        layout.yaxis['title']='voltage (mV)'
        with out2:
            clear_output()
            traces=[]
            columns=[]
            trace = go.Scatter(x=timeVectorP,y=model_current,mode = 'lines', \
                               name = 'voltage trace')
            traces.append(trace)
            columns.append(timeVectorP)
            columns.append(model_current)
            for i in range(len(TM.value)):
                traceW = go.Scatter(x=timeVectorP,y=ncweightsP[i],mode = 'lines', \
                                    name = 'synapse '+str(TM.value[i]))
                traces.append(traceW)
                columns.append(ncweightsP[i])
            iplot(go.Figure(data=traces, layout=layout))
    
    dateTimeObj = datetime.now()
    global timestampStr
    timestampStr = dateTimeObj.strftime("%Y-%b-%d-%H-%M-%S")
    with open(expprot.value+SYNC.value+timestampStr+'.csv', 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(*columns))
    Rec2.layout.display=''

Rec = widgets.Button()
Rec.description = 'Record'
Rec.background_color='gainsboro'
Rec.border_color='black'
Rec.width='30%'
Rec.on_click(play_prot)
Rec2 = widgets.Button()
Rec2.description = 'Save the traces to storage'
Rec2.background_color='gainsboro'
Rec2.border_color='black'
Rec2.width='30%'
Rec2.on_click(movetostorage)
Rec2.layout.display='none'
out2 = widgets.Output()

RS=widgets.Dropdown(
    options=[],
    description='recording site:',
    disabled=False,
    style={'description_width': 'initial'}
)

SPACE0=widgets.HTML("""<input name="mioTesto" type="text" value=" " style="color: black; background-color: white; border: none; font-weight: bold;text-align: center" height="40%" disabled="true" id="testo" />""")
SPACE4=widgets.HTML("""<input name="mioTesto" type="text" value=" " style="color: black; background-color: white; border: none; font-weight: bold;text-align: center" size="2%" disabled="true" id="testo" />""")
SPACEL=Label(' ',layout=Layout(width='13%'))
SPACEL.layout.display='none'
SPACE5=Label('Intra burst',layout=Layout(width='10%'))
SPACE5.layout.display='none'
SPACE6=Label('Inter burst',layout=Layout(width='13%'))
SPACE6.layout.display='none'
from ipywidgets import HBox, Label, Layout
L1=Label('del',layout=Layout(width='13%'))
L2=Label('dur',layout=Layout(width='13%'))
L3=Label('amp',layout=Layout(width='13%'))
L4=Label('repetitions',layout=Layout(width='13%'))
L5=Label('Netstim freq',layout=Layout(width='13%'))
L5.layout.display='none'
L6=Label('Netstim nr',layout=Layout(width='13%'))
L6.layout.display='none'
L7=Label('Vrev',layout=Layout(width='13%'))
L8=Label('VClamp amp',layout=Layout(width='13%'))
L9=Label('dur',layout=Layout(width='13%'))
accordion = widgets.Accordion(children=[widgets.VBox([RBM,file_widget,boxmodfiles,\
                                                      boxdescr,LINK],\
                                                     layout=Layout(display='flex',flex_flow='column',\
                                                                   justify_content='space-between')), \
                                        widgets.VBox([HBox([labelsynnr,synnr]),\
                                                      HBox([labelmindist,mindist]),\
                                                      HBox([labelmaxdist,maxdist]),\
                                                      HBox([APIC,labelAPIC]),\
                                                      HBox([DEND,labelDEND])]),\
                                        widgets.VBox([expprot,SYNC,\
                                                      HBox([labelnetstimintv,netstimintv]),\
                                                      HBox([labelnetstimnr,netstimnr]),\
                                                      HBox([SPACEL,SPACE5,SPACE6]),\
                                                      HBox([L5,netstimintv1,netstimintv2]),\
                                                      HBox([L6,netstimnr1,netstimnr2]),\
                                                      SPACE0,\
                                                      HBox([labelnetstimstart,netstimstart]),\
                                                      HBox([labelnetstimnoise,netstimnoise]),\
                                                      HBox([labelnetconweight,netconweight])]),\
                                       widgets.VBox([HBox([L1, stimdelay]),\
                                                     HBox([L2, stimdur]),\
                                                     HBox([L3, stimamp]),\
                                                     HBox([L4, stimrep])]),\
                                       widgets.VBox([HBox([VCLAMPrec, labelVCLAMP]),\
                                                     HBox([L7, vclampvrev]),\
                                                     HBox([L8, vclampamp]),\
                                                     HBox([L9, vclampdur])])])
accordion.set_title(0, 'MOD FILE')
accordion.set_title(1, 'SYNAPSES')
accordion.set_title(2, 'CONNECTIONS')
accordion.set_title(3, 'EXTERNAL CURRENT STIMULATIONS')
accordion.set_title(4, 'VOLTAGE CLAMP')
menu = widgets.Tab(children=[VBox(children=[tstop,accordion,SYN,outputHTML,out1]),\
                             VBox(children=[wht,RS,TM,Rec,Rec2,simnr,out2])])
menu.set_title(0, 'Config')
menu.set_title(1, 'SIM')

def morphnotshow(_):
    if menu.selected_index==1:
        noclear=1
        #clear_output()
    else:
        if menu.selected_index==0:
            #clear_output()
            if firsttimeplacesyn==2:
                with out1:
                    clear_output()
                    fig = plt.figure(figsize=(6, 6))
                    ax = Axis3DWithNEURON(fig)
                    ax.grid(False)
                    ax.axis('off')
                    ax.shapeplot(variable='v')
                    for i in range(len(pozs)):
                        ax.mark(eval('testcell'+\
                                     pozs[i].name()[pozs[i].name().find('].')+1:100]+\
                                     '('+str(listall[i][1])+')'),marker='oc',number=i)
                    ax.auto_aspect()
            else:
                with out1:
                    clear_output()
                    fig = plt.figure(figsize=(6, 6))
                    ax = Axis3DWithNEURON(fig)
                    ax.grid(False)
                    ax.axis('off')
                    ax.shapeplot(variable='v')
                    ax.auto_aspect()
#menu.observe(morphnotshow)
menu

In [ ]:
from IPython.display import HTML
HTML('''
<form action="javascript:code_toggle()">
<input id="toggle-code-button" type="submit" value="Click here to toggle on/off the raw code."></form>
<script>
code_show=true;
console.log(code_show)
$('.widget-area .prompt .close').hide()
function code_toggle() {
 console.log(code_show)
 var list = document.getElementsByClassName('cell code_cell rendered unselected');
 var list_index = []
 for (var i = 0; i < list.length; i++) {
    var curr_des = [];
    curr_wrapper = $(list[i]).find(".output_wrapper");
    curr_area = $(list[i]).find(".output_area");
    if (curr_wrapper.length > 0 && curr_area.length == 0){
    list_index.push(i)
    } else if (curr_wrapper.length == 0 && curr_area.length == 0){
    list_index.push(i)
    };
 }
 console.log(list.length)
 console.log(list_index)
 for (var j = 0; j< list_index.length; j++){
 console.log(list[list_index[j]].style.display)
 }
 if (code_show){
 $('div.input').hide();
 for (var j = 0; j< list_index.length; j++){
    list[list_index[j]].style.display = "none"
    }
 } else {
 $('div.input').show();
 for (var j = 0; j< list_index.length; j++){
    list[list_index[j]].style.display = "block"
    }
 }
 code_show = !code_show
 console.log(code_show)

 }
document.getElementById("toggle-code-button").click(); 
</script>
''')